## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nome,US,64.5011,-165.4064,49.15,93,90,8.05,light rain
1,1,Glyadyanskoye,RU,54.9075,65.0883,56.98,59,76,5.82,broken clouds
2,2,Nikolskoye,RU,59.7035,30.7861,76.89,67,25,3.85,scattered clouds
3,3,Vanimo,PG,-2.6741,141.3028,75.96,84,10,4.68,clear sky
4,4,New Norfolk,AU,-42.7826,147.0587,53.10,81,98,3.00,light rain


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))



What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Nikolskoye,RU,59.7035,30.7861,76.89,67,25,3.85,scattered clouds
3,3,Vanimo,PG,-2.6741,141.3028,75.96,84,10,4.68,clear sky
5,5,Nantucket,US,41.2835,-70.0995,83.25,85,1,11.50,clear sky
7,7,Albany,US,42.6001,-73.9662,86.58,53,1,1.99,clear sky
9,9,Mersing,MY,2.4312,103.8405,78.12,89,70,6.93,broken clouds
...,...,...,...,...,...,...,...,...,...,...
689,689,Mount Pleasant,US,32.7941,-79.8626,90.00,78,40,1.99,scattered clouds
690,690,Ribeira Brava,PT,32.6500,-17.0667,81.01,81,23,2.33,few clouds
695,695,Dubova,RO,44.6167,22.2667,78.28,60,70,6.98,broken clouds
696,696,Hazorasp,UZ,41.3194,61.0742,76.84,19,0,20.42,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                248
City                   248
Country                248
Lat                    248
Lng                    248
Max Temp               248
Humidity               248
Cloudiness             248
Wind Speed             248
Current Description    248
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Nikolskoye,RU,76.89,scattered clouds,59.7035,30.7861,
3,Vanimo,PG,75.96,clear sky,-2.6741,141.3028,
5,Nantucket,US,83.25,clear sky,41.2835,-70.0995,
7,Albany,US,86.58,clear sky,42.6001,-73.9662,
9,Mersing,MY,78.12,broken clouds,2.4312,103.8405,
12,Ribeira Grande,PT,79.20,few clouds,38.5167,-28.7000,
14,Kapaa,US,82.72,scattered clouds,22.0752,-159.3190,
15,Vaini,TO,75.45,light rain,-21.2000,-175.2000,
19,Jamestown,US,83.30,few clouds,42.0970,-79.2353,
24,Butaritari,KI,82.33,overcast clouds,3.0707,172.7902,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Nearby Places API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    print(index)
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        

2
3
5
7
Hotel not found... skipping.
9
12
14
15
19
24
26
29
30
33
38
39
41
Hotel not found... skipping.
45
59
60
65
67
71
79
83
84
86
87
89
90
99
100
105
106
112
115
117
124
126
128
133
135
136
Hotel not found... skipping.
141
142
143
146
149
150
159
161
169
170
172
174
176
178
179
181
189
192
194
195
201
202
Hotel not found... skipping.
209
215
218
219
225
228
230
235
241
245
246
248
Hotel not found... skipping.
252
253
261
262
263
266
271
274
275
279
280
282
284
285
286
289
293
294
297
301
302
303
304
305
308
313
314
324
325
328
332
336
341
343
346
348
353
355
362
367
369
370
372
374
377
379
381
382
383
384
385
386
391
Hotel not found... skipping.
397
400
401
406
410
412
414
416
421
423
424
428
429
431
435
443
448
449
454
457
Hotel not found... skipping.
458
459
462
463
464
465
467
469
470
472
473
474
475
479
480
485
486
487
488
497
503
505
506
508
509
511
512
516
517
518
520
521
522
523
524
525
527
529
530
533
Hotel not found... skipping.
534
539
540
549
551
555
558
Hotel not found.

In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Nikolskoye,RU,76.89,scattered clouds,59.7035,30.7861,Tourist House - Sablino
3,Vanimo,PG,75.96,clear sky,-2.6741,141.3028,Vahmoneh Lodge Vanimo
5,Nantucket,US,83.25,clear sky,41.2835,-70.0995,Jared Coffin House
9,Mersing,MY,78.12,broken clouds,2.4312,103.8405,Timotel Hotel
12,Ribeira Grande,PT,79.20,few clouds,38.5167,-28.7000,Quinta da Meia Eira
14,Kapaa,US,82.72,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
15,Vaini,TO,75.45,light rain,-21.2000,-175.2000,Keleti Beach Resort
19,Jamestown,US,83.30,few clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
24,Butaritari,KI,82.33,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
26,Lazaro Cardenas,MX,87.13,broken clouds,17.9583,-102.2000,quinta antigua


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [22]:
import matplotlib.pyplot as plt
#11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Add a heat layer for each city if needed.
#heat_layer = gmaps.heatmap_layer(locations,
             #dissipating=False, max_intensity=300, point_radius=4)
#fig.add_layer(heat_layer)


# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))